# MCP (Model Context Protocol) Demo

This notebook demonstrates how to use MCP with both .NET and Python implementations.

## Architecture Overview

```
┌─────────────────────────────────────────────────────────────────────┐
│         Sample REST API (Port 5001)                                 │
│  .NET: Examples/SampleRestApi/                                      │
│  Python: examples/sample_rest_api.py                                │
│  - /api/employees    - /api/products                                │
│  - /api/orders       - /api/inventory                               │
└───────────────────────▲────────────────────────────────────────────┘
                        │
                        │ HTTP calls
                        │
┌───────────────────────┴────────────────────────────────────────────┐
│              MCP Server (Port 5050)                                 │
│  .NET: Examples/SampleMcpBridge/ or McpServer/                      │
│  Python: examples/mcp_server.py                                     │
│  - /tools           - List available MCP tools                      │
│  - /tools/{name}/call - Execute MCP tool                           │
│  - /sse             - SSE transport endpoint                        │
└───────────────────────▲────────────────────────────────────────────┘
                        │
                        │ MCP Protocol
                        │
┌───────────────────────┴────────────────────────────────────────────┐
│         Agent Framework (Port 8000)                                 │
│  .NET: dotnet run                                                   │
│  Python: uvicorn main:app --port 8000                               │
│  - McpClientService connects to MCP servers                         │
│  - AI agents can call MCP tools                                     │
└────────────────────────────────────────────────────────────────────┘
```

## Quick Start Commands

### Option A: All Python
```bash
# Terminal 1: Sample REST API
cd Backend/python/examples
python sample_rest_api.py

# Terminal 2: MCP Server
cd Backend/python/examples
python mcp_server.py --port 5050

# Terminal 3: Agent Framework
cd Backend/python
uvicorn main:app --port 8000
```

### Option B: All .NET
```bash
# Terminal 1: Sample REST API
cd Backend/dotnet/Examples/SampleRestApi
dotnet run

# Terminal 2: MCP Bridge
cd Backend/dotnet/Examples/SampleMcpBridge
dotnet run

# Terminal 3: Agent Framework
cd Backend/dotnet
dotnet run
```

In [ ]:
import requests
import json
from datetime import datetime

# Configuration
SAMPLE_REST_API_URL = "http://localhost:5001"  # Sample REST API
MCP_SERVER_URL = "http://localhost:5050"       # MCP Server
AGENT_FRAMEWORK_URL = "http://localhost:8000"  # Agent Framework

headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer demo-token-12345"
}

def pretty_print(data):
    """Pretty print JSON data"""
    print(json.dumps(data, indent=2, default=str))

def check_service(name, url, endpoint="/health"):
    """Check if a service is running"""
    try:
        full_url = f"{url}{endpoint}"
        resp = requests.get(full_url, timeout=5)
        if resp.status_code == 200:
            print(f"✅ {name} is running at {url}")
            return True
        else:
            print(f"⚠️  {name} returned status {resp.status_code}")
            return False
    except requests.exceptions.ConnectionError:
        print(f"❌ {name} is not running at {url}")
        return False
    except Exception as e:
        print(f"❌ {name} error: {e}")
        return False

print("MCP Demo Notebook Initialized")
print(f"Sample REST API: {SAMPLE_REST_API_URL}")
print(f"MCP Server: {MCP_SERVER_URL}")
print(f"Agent Framework: {AGENT_FRAMEWORK_URL}")

## 1. Check Services Health

In [ ]:
print("Checking services...\n")

rest_api_running = check_service("Sample REST API", SAMPLE_REST_API_URL, "/api/health")
mcp_server_running = check_service("MCP Server", MCP_SERVER_URL, "/health")
agent_framework_running = check_service("Agent Framework", AGENT_FRAMEWORK_URL, "/health")

print("\n" + "="*60)
if rest_api_running and mcp_server_running:
    print("✅ Full MCP stack is running!")
elif rest_api_running:
    print("⚠️  REST API running but MCP Server not running")
    print("    MCP tools will use HTTP fallback directly to REST API")
elif agent_framework_running:
    print("⚠️  Only Agent Framework running")
    print("    Can use MCP simulation endpoint")
else:
    print("❌ No services running")
    print("\nStart services with:")
    print("  Python REST API: cd Backend/python/examples && python sample_rest_api.py")
    print("  Python MCP Server: cd Backend/python/examples && python mcp_server.py")
    print("  Python Agent Framework: cd Backend/python && uvicorn main:app --port 8000")

## 2. Direct REST API Calls

First, let's call the Sample REST API directly to see the data.

In [ ]:
# Get all employees from REST API
if rest_api_running:
    response = requests.get(f"{SAMPLE_REST_API_URL}/api/employees")
    print("Employees from REST API:")
    pretty_print(response.json())
else:
    print("❌ Sample REST API not running")

In [ ]:
# Get all products from REST API
if rest_api_running:
    response = requests.get(f"{SAMPLE_REST_API_URL}/api/products")
    print("Products from REST API:")
    pretty_print(response.json())
else:
    print("❌ Sample REST API not running")

In [ ]:
# Get inventory from REST API
if rest_api_running:
    response = requests.get(f"{SAMPLE_REST_API_URL}/api/inventory")
    print("Inventory from REST API:")
    pretty_print(response.json())
else:
    print("❌ Sample REST API not running")

## 3. MCP Server - List Tools

The MCP Server exposes REST API endpoints as MCP tools.

In [ ]:
# List available MCP tools
if mcp_server_running:
    response = requests.get(f"{MCP_SERVER_URL}/tools")
    data = response.json()
    print(f"Available MCP Tools ({data.get('total', 0)} tools):\n")
    for tool in data.get("tools", []):
        print(f"  📦 {tool['name']}")
        print(f"     {tool['description']}")
        print()
else:
    print("❌ MCP Server not running")
    print("   Start with: python examples/mcp_server.py --port 5050")

## 4. Call MCP Tools Directly

Call MCP tools through the MCP Server's HTTP API.

In [ ]:
# Call get_all_employees via MCP Server
if mcp_server_running:
    response = requests.post(
        f"{MCP_SERVER_URL}/tools/get_all_employees/call",
        json={"arguments": {}}
    )
    print("MCP Tool: get_all_employees")
    pretty_print(response.json())
else:
    print("❌ MCP Server not running")

In [ ]:
# Call search_employees via MCP Server
if mcp_server_running:
    response = requests.post(
        f"{MCP_SERVER_URL}/tools/search_employees/call",
        json={"arguments": {"department": "Engineering"}}
    )
    print("MCP Tool: search_employees(department='Engineering')")
    pretty_print(response.json())
else:
    print("❌ MCP Server not running")

In [ ]:
# Call get_products_by_category via MCP Server
if mcp_server_running:
    response = requests.post(
        f"{MCP_SERVER_URL}/tools/get_products_by_category/call",
        json={"arguments": {"category": "Electronics"}}
    )
    print("MCP Tool: get_products_by_category(category='Electronics')")
    pretty_print(response.json())
else:
    print("❌ MCP Server not running")

In [ ]:
# Call create_order via MCP Server
if mcp_server_running:
    response = requests.post(
        f"{MCP_SERVER_URL}/tools/create_order/call",
        json={"arguments": {"customer_id": 1, "product_ids": [1, 2]}}
    )
    print("MCP Tool: create_order(customer_id=1, product_ids=[1,2])")
    pretty_print(response.json())
else:
    print("❌ MCP Server not running")

## 5. Agent Framework - MCP Integration

The Agent Framework connects to MCP servers and exposes their tools.

In [ ]:
# Get configured MCP servers
if agent_framework_running:
    response = requests.get(f"{AGENT_FRAMEWORK_URL}/api/mcp/servers", headers=headers)
    print("Configured MCP Servers:")
    pretty_print(response.json())
else:
    print("❌ Agent Framework not running")

In [ ]:
# Get all MCP tools via Agent Framework
if agent_framework_running:
    response = requests.get(f"{AGENT_FRAMEWORK_URL}/api/mcp/tools", headers=headers)
    data = response.json()
    print(f"MCP Tools via Agent Framework ({data.get('total', 0)} tools):")
    pretty_print(response.json())
else:
    print("❌ Agent Framework not running")

In [ ]:
# Call MCP tool through Agent Framework
if agent_framework_running:
    response = requests.post(
        f"{AGENT_FRAMEWORK_URL}/api/mcp/servers/python-mcp-server/tools/get_all_employees/call",
        json={"arguments": {}},
        headers=headers
    )
    print("MCP Tool via Agent Framework:")
    result = response.json()
    print(f"Success: {result.get('success')}")
    print(f"Duration: {result.get('durationMs')}ms")
    if result.get('success') and result.get('result'):
        try:
            pretty_print(json.loads(result['result']))
        except:
            print(result['result'])
    elif result.get('error'):
        print(f"Error: {result.get('error')}")
else:
    print("❌ Agent Framework not running")

## 6. MCP Simulation (No External Services Needed)

The Agent Framework includes an MCP simulation endpoint for testing.

In [ ]:
# Simulate get_all_employees
if agent_framework_running:
    response = requests.post(
        f"{AGENT_FRAMEWORK_URL}/api/mcp/demo/simulate",
        json={"operation": "get_all_employees", "arguments": {}},
        headers=headers
    )
    print("Simulated MCP: get_all_employees")
    pretty_print(response.json())
else:
    print("❌ Agent Framework not running")

In [ ]:
# Simulate search_employees
if agent_framework_running:
    response = requests.post(
        f"{AGENT_FRAMEWORK_URL}/api/mcp/demo/simulate",
        json={
            "operation": "search_employees",
            "arguments": {"department": "Engineering"}
        },
        headers=headers
    )
    print("Simulated MCP: search_employees(Engineering)")
    pretty_print(response.json())
else:
    print("❌ Agent Framework not running")

In [ ]:
# Simulate get_products_by_category
if agent_framework_running:
    response = requests.post(
        f"{AGENT_FRAMEWORK_URL}/api/mcp/demo/simulate",
        json={
            "operation": "get_products_by_category",
            "arguments": {"category": "Electronics"}
        },
        headers=headers
    )
    print("Simulated MCP: get_products_by_category(Electronics)")
    pretty_print(response.json())
else:
    print("❌ Agent Framework not running")

In [ ]:
# Simulate create_order
if agent_framework_running:
    response = requests.post(
        f"{AGENT_FRAMEWORK_URL}/api/mcp/demo/simulate",
        json={
            "operation": "create_order",
            "arguments": {
                "customer_id": 1,
                "product_id": 2,
                "quantity": 3
            }
        },
        headers=headers
    )
    print("Simulated MCP: create_order")
    pretty_print(response.json())
else:
    print("❌ Agent Framework not running")

In [ ]:
# Simulate get_inventory
if agent_framework_running:
    response = requests.post(
        f"{AGENT_FRAMEWORK_URL}/api/mcp/demo/simulate",
        json={"operation": "get_inventory", "arguments": {}},
        headers=headers
    )
    print("Simulated MCP: get_inventory")
    pretty_print(response.json())
else:
    print("❌ Agent Framework not running")

## 7. Demo API (Built into Agent Framework)

The Agent Framework also has built-in demo endpoints.

In [ ]:
# Get weather from Demo API
if agent_framework_running:
    response = requests.get(f"{AGENT_FRAMEWORK_URL}/api/demo/weather?location=Seattle")
    print("Demo API: Weather for Seattle")
    pretty_print(response.json())
else:
    print("❌ Agent Framework not running")

In [ ]:
# Get products from Demo API
if agent_framework_running:
    response = requests.get(f"{AGENT_FRAMEWORK_URL}/api/demo/products")
    print("Demo API: Products")
    pretty_print(response.json())
else:
    print("❌ Agent Framework not running")

## Summary

### Python Files

| File | Description | Port |
|------|-------------|------|
| `examples/sample_rest_api.py` | Business REST API | 5001 |
| `examples/mcp_server.py` | MCP Server (SSE/STDIO) | 5050 |
| `main.py` | Agent Framework | 8000 |

### .NET Files

| Project | Description | Port |
|---------|-------------|------|
| `Examples/SampleRestApi/` | Business REST API | 5001 |
| `Examples/SampleMcpBridge/` | MCP Bridge (SSE) | 5050 |
| `McpServer/` | MCP Server (STDIO) | - |
| Main project | Agent Framework | 8000 |

### MCP Tools Available

- `get_all_employees` - List all employees
- `get_employee` - Get employee by ID
- `search_employees` - Search by name/department
- `get_all_products` - List all products
- `get_product` - Get product by ID
- `get_products_by_category` - Filter by category
- `get_all_orders` - List all orders
- `get_order` - Get order by ID
- `create_order` - Create new order
- `get_inventory` - Get stock status
- `update_stock` - Update product stock
- `check_api_health` - Health check